# Tile Generation from GeoTIFF Images

In this notebook, a Python script designed to facilitate the generation of smaller tiles from large GeoTIFF images. The process involves partitioning the images into manageable sections, often referred to as tiles, which can be useful for various applications such as processing large datasets in smaller chunks or serving imagery over the web efficiently.

In [2]:
# import all necessary libraries
import os
from itertools import product
import rasterio as rio
from rasterio import windows

Now we will read GeoTIFF images from a specified input directory (in_path), divide them into smaller tiles, and save these tiles to an output directory (out_path).

In [ ]:
in_path = 'D:\\images\\'
# input_filename = 'image.tif'
 
out_path = 'D:\\5_5_patches\\'
# output_filename = 'tile_{}-{}.tif'
w=0
def get_tiles(ds, width=5, height=5):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform

image_files = [f for f in os.listdir(in_path) if f.endswith('.tif')]
for file in image_files:
    with rio.open(os.path.join(in_path+file)) as inds:
        tile_width, tile_height =5,5

        meta = inds.meta.copy()

        for window, transform in get_tiles(inds):
            print(window)
            meta['transform'] = transform
            meta['width'], meta['height'] = window.width, window.height
            outpath = os.path.join(out_path,format(w)+file.format(int(window.col_off), int(window.row_off)))
            with rio.open(outpath, 'w', **meta) as outds:
                outds.write(inds.read(window=window))
                w=w+1